---
cdt: 2024-09-10T16:27:00
title: Creating the 'clean' Schema and Primary Keys
project: database_architecture
description: Introduces the 'clean' schema and provides the code necessary for its creation from the source tables. A defining characteristic is the presence of 'pk' primary keys in each table
conclusion: A 'clean' schema was created with the tables 'chm', 'st', 'ct' with primary keys 'pk' based on a unique identifying column. Rational constraints were added for each table. 'pk_chm' was added to 'pbl.chromatogram_spectra_long'. 'sample_num' was recreated in 'clean.chm' to link that table to 'pbl.sample_metadata'.
---

In [ ]:
%reload_ext autoreload
%autoreload 2
import duckdb as db
import polars as pl
from pca_analysis.experiments.constants import db_path
from great_tables import GT
pl.Config.set_tbl_rows(999).set_tbl_width_chars(2000).set_fmt_str_lengths(99999)

con = db.connect(db_path)
con.close()


[Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture) describes a tiered database structure from the raw, dirty, messy input data to cleaned, organised and human readable outputs. From this point of view, our data is still very much in the lower stages. In order to begin organising the data in a more cohesive manner, each table needs a contextless primary key. The addition of the primary key column ('pk') to each of the 'c_chemstation_metadata', 'c_sample_tracker' and 'c_cellar_tracker' will begin this second-stage cleaning process, and the new tables will be added to a newly created 'clean' schema. The advantage of the distinction is that it a: saves on namespace, and b. protects the source data tables from incorrect edits.

To produce the primary keys we identify columns that can be used as unique identifiers, then use `dense_rank()` over them to produce a unique, monotonically increasing key column. This was done for each table. See below for details.


# Create 'clean' Schema

Schemas need to be explicitly created before tables can be added to them.

In [ ]:
con.sql(
"""--sql
CREATE SCHEMA IF NOT EXISTS clean;
""")


# Adding Primary Keys and Creating 'clean.' Tables



## Sample Tracker


### Observing 'samplecode' as a Primary Key


is ch_samplecode the primary key of sample tracker? If it is, there should be an equal number of distinct values as there are rows of the table.

In [ ]:
con.sql(
"""--sql
SELECT
    samplecode,
    count(*) as count
FROM
    c_sample_tracker
GROUP BY
    samplecode
HAVING
    count > 1
"""
).pl()


yes. However lets add a new 'pk' column based on it. This can be achieved by densely ranking the 'samplecode' column:

In [ ]:
con.sql(
"""--sql

"""
)


Now I have to recreate the table.


Now I have to create the clean schema table:


In [ ]:
con.sql(
"""--sql
SELECT * FROM st LIMIT 1
"""
).pl().columns


In [ ]:
con.sql(
"""--sql
-- create a intermediate table with the primary key to avoid editing the source
CREATE TEMP TABLE st AS
    SELECT
        *,
        rank_dense() OVER (order by samplecode) as pk
    FROM
        c_sample_tracker;

-- create the destination table with the primary key column constraint
CREATE OR REPLACE TABLE clean.st (
    detection VARCHAR,
    sampler VARCHAR,
    samplecode VARCHAR,
    vintage VARCHAR,
    name VARCHAR,
    open_date VARCHAR,
    sampled_date VARCHAR,
    added_to_cellartracker VARCHAR,
    notes VARCHAR,
    size VARCHAR,
    ct_wine_name VARCHAR,
    pk INTEGER PRIMARY KEY,
);

-- populate the new table
INSERT INTO clean.st
SELECT
    *
FROM
    st;

-- cleanup of the intermedaite table
DROP TABLE st;
SELECT
    *
FROM
    clean.st
LIMIT 3
"""
).pl()


And see if the primary key constraint worked..

If the above passed, then the primary key constraint on sample tracker worked.

## Chemstation Metadata

The chemstation metadata primary key can be created from the 'id' column.

To do this we create a new 'clean' table,



The primary key of the 'chemstation_metadata' table is the 'id'.


There are 175 unique rows.


create the pk column


In [ ]:
if not con.sql(
"""--sql
SELECT
    id,
    count(*) as count
FROM
    c_chemstation_metadata
GROUP BY
    id
HAVING
    count > 1
"""
).pl().is_empty():
    raise ValueError("duplicate detected in table")


In [ ]:
con.sql(
"""--sql
-- create an intermediate table to avoid editing the source dat and add the primary key
CREATE OR REPLACE TEMP TABLE chm AS
SELECT
    rank_dense() OVER (order by id) as pk,
    id,
    path,
    acq_date,
    acq_method,
    unit,
    signal,
    vendor,
    inj_vol,
    seq_name,
    seq_desc,
    vialnum,
    originalfilepath,
    "desc",
    join_samplecode as st_samplecode,
FROM
    c_chemstation_metadata;

-- create the destination table with the primary key restraint
CREATE OR REPLACE TABLE clean.chm (
    pk INTEGER PRIMARY KEY,
    id VARCHAR UNIQUE,
    path VARCHAR,
    acq_date VARCHAR UNIQUE,
    acq_method VARCHAR,
    unit VARCHAR,
    signal VARCHAR,
    vendor VARCHAR,
    inj_vol VARCHAR,
    seq_name VARCHAR,
    seq_desc VARCHAR,
    vialnum VARCHAR,
    originalfilepath VARCHAR,
    st_samplecode VARCHAR,
    "desc" VARCHAR,
);

-- populate the destination table
INSERT INTO clean.chm
    SELECT
        pk,
        id,
        path,
        acq_date,
        acq_method,
        unit,
        signal,
        vendor,
        inj_vol,
        seq_name,
        seq_desc,
        vialnum,
        originalfilepath,
        st_samplecode,
        "desc"
    FROM
        chm;

-- cleanup
DROP TABLE chm;

SELECT * FROM clean.chm LIMIT 3
"""
).pl()


And now to populate. I think i'll have to do this column by column? Need a join table

Now write chemstation metadata to the clean schema


### Clean CHM Description


In [ ]:
con.sql(
"""--sql
DESCRIBE clean.chm
"""
).pl()


### Constraints Table


In [ ]:
con.sql(
"""--sql
SELECT
    schema_name,
    table_name,
    constraint_column_names,
    constraint_type
FROM
    duckdb_constraints()
WHERE
    schema_name = 'clean'
AND
    table_name = 'chm'
"""
).pl()


Now it would acquire the means to connect to the sample tracker table..

In [ ]:
join_tbl = con.sql(
"""--sql
SELECT
    st.ct_wine_name as pk_st_to_ct,
    jt.pk_chm_to_st,
    jt.pk_id
FROM
    c_sample_tracker as st
JOIN
    join_tbl as jt
ON
    jt.pk_chm_to_st = st.samplecode
"""
).pl()

join_tbl.head()


In [ ]:
con.sql(
"""--sql
SELECT
    *
FROM
    
"""
).pl()


In [ ]:
con.sql(
"""--sql
select table_schema, table_name, column_name from information_schema.columns WHERE table_name = 'c_sample_tracker'
"""
).pl()['column_name'].to_list()


In [ ]:
con.sql(
"""--sql
SELECT
    wine
FROM
    c_cellar_tracker
"""
).pl()


In [ ]:
con.sql(
"""--sql
SELECT
    ct_wine_name
FROM
    c_sample_tracker
"""
).pl()
con.s


'c_chemstation_metadata' has 175 rows but 174 distinct 'join_samplecode' values. This is the key to join with 'c_sample_tracker'. Is there a duplicate?

In [ ]:
con.sql(
"""--sql
select
    count(distinct join_samplecode)
from
    c_chemstation_metadata
"""
).pl()


In [ ]:
con.sql(
"""--sql
select
    *
from
    c_cellar_tracker as ct
JOIN
    c_sample_tracker as st
ON
    ct.wine = st.ct_wine_name
"""
).pl()


In [ ]:
con.sql(
"""--sql
SELECT
    vintage,
    count(vintage) as count
FROM
    pbl.sample_metadata
JOIN
    c_a
    
"""
).pl()


## Cellar Tracker

To form a join table between ST and CT it is best to create a new primary key on cellar tracker (CT).


### CT Duplicate Row


CT has one duplicate row, wine 'Mumm Tasmania Brut Prestige'.

In [ ]:
con.sql(
"""--sql
SELECT
    vintage,
    name,
    count(*) as duplicate_count
FROM
    c_cellar_tracker
GROUP BY
    name, vintage
HAVING
    count(*) > 1
"""
).pl()


This can be remedied by selecting distinct rows only in the query.

In [ ]:
con.sql(
"""--sql
-- create an intermediate table including the primary key to avoid editing the source table.
CREATE OR REPLACE TEMP TABLE ct AS
    SELECT
        size,
        vintage,
        name,
        locale,
        country,
        region,
        subregion,
        appellation,
        producer,
        type,
        color,
        category,
        varietal,
        wine,
        rank_dense() OVER (order by vintage, name) as pk,
    FROM
        (
            SELECT
                -- remove duplicate row from consideration - Mumm NV is duplicated
                DISTINCT concat(vintage, name),
                size,
                vintage,
                name,
                locale,
                country,
                region,
                subregion,
                appellation,
                producer,
                type,
                color,
                category,
                varietal,
                wine,
            FROM
                c_cellar_tracker
            );

-- create the destination table
CREATE OR REPLACE TABLE clean.ct (
    size VARCHAR,
    vintage VARCHAR,
    name VARCHAR,
    locale VARCHAR,
    country VARCHAR,
    region VARCHAR,
    subregion VARCHAR,
    appellation VARCHAR,
    producer VARCHAR,
    type VARCHAR,
    color VARCHAR,
    category VARCHAR,
    varietal VARCHAR,
    wine VARCHAR,
    pk INTEGER PRIMARY KEY
);

-- populate the destination table
INSERT INTO clean.ct (
SELECT
    size,
    vintage,
    name,
    locale,
    country,
    region,
    subregion,
    appellation,
    producer,
    type,
    color,
    category,
    varietal,
    wine,
    pk
FROM
    ct
);

-- cleanup
DROP TABLE ct;

SELECT * FROM clean.ct LIMIT 3
"""
).pl()


# Add CHM's PK to CS

While the 'id' column is acceptable, in the spirit of simplying the keys, we can add the CHM primary key 'pk' to the 'pbl.chromatogram_spectra_long' table as well.

In [ ]:
con.sql(
"""--sql
ALTER TABLE
    pbl.chromatogram_spectra_long
ADD COLUMN IF NOT EXISTS
    pk_chm INTEGER;
UPDATE pbl.chromatogram_spectra_long as cs
SET
    pk_chm = (
    SELECT
        pk
    FROM
        clean.chm as chm
    WHERE
        cs.id = chm.id
    );
SELECT * FROM pbl.chromatogram_spectra_long LIMIT 3;
"""
).pl()


# Adding 'sample_num'

Moving from 'pbl' schema to 'clean' schema creates one problem - all the old code will reference 'sample_num' as the primary key rather than 'pk_chm', for example. To remedy this, we can add the 'sample_num' column to 'clean.chm'. First we'll create a test to ensure that the method of value creation is the same as the original.

In [ ]:
con.sql(
"""--sql
WITH chm AS (SELECT
    rank_dense() OVER (ORDER BY acq_date) as sample_num_new,
    id
FROM
    clean.chm as chm)
SELECT
    bool_and(sample_num_new = sm.sample_num) as all_same
FROM
    chm
JOIN
    pbl.sample_metadata as sm
USING
    (id)
"""
).pl()


As the above is 'true', then it is. Thus we can create the column in 'chm'.

In [ ]:
con.sql(
"""--sql
ALTER TABLE
    clean.chm
ADD COLUMN IF NOT EXISTS
    sample_num INTEGER;
CREATE OR REPLACE TEMP TABLE sample_num AS (
    SELECT
        rank_dense() OVER (ORDER BY acq_date) as sample_num,
        pk
    FROM
        clean.chm
    );
UPDATE clean.chm
SET
    sample_num = (
        SELECT
            sample_num
        FROM
            sample_num
        WHERE
            clean.chm.pk = sample_num.pk
            );
SELECT
    pk,
    acq_date,
    sample_num
FROM
    clean.chm
ORDER BY
    acq_date
LIMIT 5
"""
).pl()


# Conclusion

In conclusion, the following tables were created in the 'clean' schema: 1. 'clean.st', 2. 'clean.chm', and 'clean.ct' corresponding to the 'c_sample_tracker', 'c_chemstation_metadata', and 'c_cellar_tracker' tables respectively. 1.'s primary key was created from the 'id' column, 2. from the 'samplecode' column and '3.' from the combination of 'vintage' and 'name' columns. In what feels like a half-done job, 'clean.chm''s primary key was also added to 'pbl.chromatogram_spectra_long' as 'pk_chm'. Furthermore, a 'sample_num' column was recreated in 'clean.chm' to provide a link to 'pbl.sample_metadata'.
